In [1]:
from logger import MyLogger
import SubjObjLoader
import json
from torch import nn
import torch
from pytorch_transformers import *
import importlib
from collections import deque
# import dataloader
from BertRDMLoader import *
import json
from torch import nn
import torch
from pytorch_transformers import *
import importlib
from tensorboardX import SummaryWriter
import torch.nn.utils.rnn as rnn_utils
import tsentiLoader

In [2]:
from emotionLoader import *
from SubjObjLoader import *
import numpy as np

### BERT RDM CM 模型代码
> 需要改动的地方
> - 损失函数要从训练函数中拆分出来，方便后面的联合训练
> 

In [3]:
class pooling_layer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(pooling_layer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def forward(self, inputs):
        assert(inputs.ndim == 4 ) # [batchsize, max_seq_len, max_word_num, input_dim] 
        batch_size, max_seq_len, max_word_num, input_dim = inputs.shape
        assert(input_dim == self.input_dim)
        t_inputs = inputs.reshape([-1, self.input_dim])
        return self.linear(t_inputs).reshape(
            
            [-1, max_word_num, self.output_dim]
        
        ).max(axis=1)[0].reshape(
        
            [-1, max_seq_len, self.output_dim]
        
        )

class RDM_Model(nn.Module):
    def __init__(self, word_embedding_dim, sent_embedding_dim, hidden_dim, dropout_prob):
        super(RDM_Model, self).__init__()
        self.embedding_dim = sent_embedding_dim
        self.hidden_dim = hidden_dim
        self.gru_model = nn.GRU(word_embedding_dim, 
                                self.hidden_dim, 
                                batch_first=True, 
                                dropout=dropout_prob
                            )
        self.DropLayer = nn.Dropout(dropout_prob)
#         self.PoolLayer = pooling_layer(word_embedding_dim, sent_embedding_dim) 
        
    def forward(self, x_emb, x_len, init_states): 
        """
        input_x: [batchsize, max_seq_len, sentence_embedding_dim] 
        x_emb: [batchsize, max_seq_len, 1, embedding_dim]
        x_len: [batchsize]
        init_states: [batchsize, hidden_dim]
        """
        batchsize, max_seq_len, _ , emb_dim = x_emb.shape
#         pool_feature = self.PoolLayer(x_emb)
#         sent_feature = sentiModel( 
#                 x_emb.reshape(
#                     [-1, max_sent_len, emb_dim]
#                 ) 
#             ).reshape(
#                 [batchsize, max_seq_len, -1]
#             )
#         pooled_input_x_dp = self.DropLayer(input_x)
        pool_feature = x_emb.reshape(
                [-1, max_seq_len, emb_dim]
        )
        df_outputs, df_last_state = self.gru_model(pool_feature, init_states)
        hidden_outs = [df_outputs[i][:x_len[i]] for i in range(batchsize)]
        final_outs = [df_outputs[i][x_len[i]-1] for i in range(batchsize)]
        return hidden_outs, final_outs


class CM_Model(nn.Module):
    def __init__(self, sentence_embedding_dim, hidden_dim, action_num):
        super(CM_Model, self).__init__()
        self.sentence_embedding_dim = sentence_embedding_dim
        self.hidden_dim = hidden_dim
        self.action_num = action_num
#         self.PoolLayer = pooling_layer(self.embedding_dim, 
#                                             self.hidden_dim)
        self.DenseLayer = nn.Linear(self.hidden_dim, 64)
        self.Classifier = nn.Linear(64, self.action_num)
        
    def forward(self, rdm_model, s_model, rl_input, rl_state):
        """
        rl_input: [batchsize, max_word_num, sentence_embedding_dim]
        rl_state: [1, batchsize, hidden_dim]
        """
        assert(rl_input.ndim==3)
        batchsize, max_word_num, embedding_dim = rl_input.shape
#         assert(embedding_dim==self.embedding_dim)
        sentence = s_model(rl_input).reshape(batch_size, 1, self.sentence_embedding_dim)
#         pooled_rl_input = self.PoolLayer(
#             rl_input.reshape(
#                 [-1, 1, max_word_num, self.embedding_dim]
#             )
#         ).reshape([-1, 1, self.hidden_dim])
        
#         print("sentence:", sentence.shape)
#         print("rl_state:", rl_state.shape)
        rl_output, rl_new_state = rdm_model.gru_model(
                                            sentence, 
                                            rl_state
                                        )
        rl_h1 = nn.functional.relu(
            self.DenseLayer(
#                 rl_state.reshape([len(rl_input), self.hidden_dim]) #it is not sure to take rl_state , rather than rl_output, as the feature
                rl_output.reshape(
                    [len(rl_input), self.hidden_dim]
                )
            )
        )
        stopScore = self.Classifier(rl_h1)
        isStop = stopScore.argmax(axis=1)
        return stopScore, isStop, rl_new_state


# In[3]:


def layer2seq(bert, layer, cuda=False):
    if cuda:
        outs = [bert( torch.tensor([input_]).cuda())
                for input_ in layer]   
    else: 
        outs = [bert( torch.tensor([input_]))
                    for input_ in layer]
    states = [item[1] for item in outs]
    return rnn_utils.pad_sequence(states, batch_first=True)

def Word_ids2SeqStates(word_ids, bert, ndim, cuda=False):
    assert(ndim == 3)
    if cuda:
        embedding = [layer2seq(bert, layer, cuda) for layer in word_ids]
    else:
        embedding = [layer2seq(bert, layer) for layer in word_ids]
    return padding_sequence(embedding)


# In[4]:


def Count_Accs(ylabel, preds):
    correct_preds = np.array(
        [1 if y1==y2 else 0 
        for (y1, y2) in zip(ylabel, preds)]
    )
    y_idxs = [idx if yl >0 else idx - len(ylabel) 
            for (idx, yl) in enumerate(ylabel)]
    pos_idxs = list(filter(lambda x: x >= 0, y_idxs))
    neg_idxs = list(filter(lambda x: x < 0, y_idxs))
    acc = sum(correct_preds) / (1.0 * len(ylabel))
    if len(pos_idxs) > 0:
        pos_acc = sum(correct_preds[pos_idxs])/(1.0*len(pos_idxs))
    else:
        pos_acc = 0
    if len(neg_idxs) > 0:
        neg_acc = sum(correct_preds[neg_idxs])/(1.0*len(neg_idxs))
    else:
        neg_acc = 0
    return acc, pos_acc, neg_acc, y_idxs, pos_idxs, neg_idxs, correct_preds

def Loss_Fn(ylabel, pred_scores):
    diff = ((ylabel - pred_scores)*(ylabel - pred_scores)).mean(axis=1)
#     pos_neg = (1.0*sum(ylabel.argmax(axis=1)))/(1.0*(len(ylabel) - sum(ylabel.argmax(axis=1))))
    pos_neg = 0
    if pos_neg > 0:
        print("unbalanced data")
        weight = torch.ones(len(ylabel)).cuda() + (ylabel.argmax(axis=1).to(torch.float32)/(1.0*pos_neg)) - ylabel.argmax(axis=1).to(torch.float32)
        return (weight *diff).mean()
    else:
        print("totally unbalanced data")
        return diff.mean()


# In[5]:
def rdm_loss(x, y, x_len, bert, rdm_model, rdm_classifier, loss_fn):
    x_emb = Word_ids2SeqStates(x, bert, 3, cuda=True) 
    batchsize, max_seq_len, max_sent_len, emb_dim = x_emb.shape
    init_states = torch.zeros([1, batchsize, rdm_model.hidden_dim]).cuda()
    rdm_hiddens, rdm_outs = rdm_model(x_emb, x_len, init_states)
    rdm_scores = rdm_classifier(
        torch.cat(
            rdm_outs # a list of tensor, where the ndim of tensor is 1 and the shape of tensor is [hidden_size]
        ).reshape(
            [-1, rdm_model.hidden_dim]
        )
    )
    rdm_preds = rdm_scores.argmax(axis=1)
    y_label = y.argmax(axis=1)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, rdm_preds)
    loss = loss_fn(rdm_scores, torch.tensor(y_label).cuda())
    return loss, acc


def TrainRDMModel(rdm_model, bert, rdm_classifier, 
                    tokenizer, t_steps, new_data_len=[], logger=None, 
                        log_dir="RDMBertTrain"):
    batch_size = 20 
    max_gpu_batch = 5 #cannot load a larger batch into the limited memory, but we could  accumulates grads
    assert(batch_size%max_gpu_batch == 0)
    sum_loss = 0.0
    sum_acc = 0.0
    t_acc = 0.9
    ret_acc = 0.0
    init_states = torch.zeros([1, 5, rdm_model.hidden_dim], dtype=torch.float32).cuda()
    weight = torch.tensor([2.0, 1.0], dtype=torch.float32).cuda()
    loss_fn = nn.CrossEntropyLoss(weight=weight)
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-5},
                                {'params': rdm_classifier.parameters(), 'lr': 5e-5},
                                {'params': rdm_model.parameters(), 'lr': 5e-5}
                             ]
    )
    
    writer = SummaryWriter(log_dir)
    acc_l = np.zeros(int(batch_size/max_gpu_batch))
    loss_l = np.zeros(int(batch_size/max_gpu_batch))
    for step in range(t_steps):
        optim.zero_grad()
        for j in range(int(batch_size/max_gpu_batch)):
            if len(new_data_len) > 0:
                x, x_len, y = get_df_batch(step, max_gpu_batch, new_data_len, tokenizer=tokenizer)
            else:
                x, x_len, y = get_df_batch(step, max_gpu_batch, tokenizer=tokenizer)
                
            loss, acc = rdm_loss(x, y, bert, rdm_model, rdm_classifier, loss_fn)
            loss.backward()
            loss_l[j] = loss
            acc_l[j] = acc
            
        optim.step()        
        writer.add_scalar('Train Loss', loss_l.mean(), step)
        writer.add_scalar('Train Accuracy', acc_l.mean(), step)

        sum_loss += loss_l.mean()
        sum_acc += acc_l.mean()
        

        if step % 10 == 9:
            sum_loss = sum_loss / 10
            sum_acc = sum_acc / 10
            print('%3d | %d , train_loss/accuracy = %6.8f/%6.7f'             % (step, t_steps, 
                sum_loss, sum_acc,
                ))
            logger.info('%3d | %d , train_loss/accuracy = %6.8f/%6.7f'             % (step, t_steps, 
                sum_loss, sum_acc,
                ))
            if step%500 == 499:
                rdm_save_as = '/home/hadoop/ERD/%s/rdmModel_epoch%03d.pkl'                                    % (log_dir, step/500, sum_acc)
                torch.save(
                    {
                        "bert":bert.state_dict(),
                        "sentiModel":sentiModel.state_dict(),
                        "rmdModel":rdm_model.state_dict(),
                        "rdm_classifier": rdm_classifierdm.state_dict()
                    },
                    rdm_save_as
                )
#                 rdm_model, bert, sentiModel, rdm_classifier
            if sum_acc > t_acc:
                break
            sum_acc = 0.0
            sum_loss = 0.0

    print(get_curtime() + " Train df Model End.")
    logger.info(get_curtime() + " Train df Model End.")
    return ret_acc


# In[6]:


def TrainCMModel(bert, rdm_model, rdm_classifier, cm_model, tokenizer, log_dir, logger, FLAGS):
    batch_size = 20
    t_acc = 0.9
    ids = np.array(range(batch_size), dtype=np.int32)
    seq_states = np.zeros([batch_size], dtype=np.int32)
    isStop = np.zeros([batch_size], dtype=np.int32)
    max_id = batch_size
    df_init_states = torch.zeros([1, batch_size, FLAGS.hidden_dim], dtype=torch.float32).cuda()
    state = df_init_states
    D = deque()
    ssq = []
    print("in RL the begining")
    rdm_optim = torch.optim.Adagrad([
                            {'params': bert.parameters(), 'lr':1e-3},
    #                                 {'params': rdm_classifier.parameters(), 'lr': 5e-2},
                            {'params': rdm_model.parameters(), 'lr': 5e-2},
                            {'params': sentiModel.parameters(), 'lr': 1e-2}
                         ],
                            weight_decay = 0.2
    )
    rl_optim = torch.optim.Adam([{'params':cm_model.parameters(), 'lr':1e-3}])
    # get_new_len(sess, mm)
    data_ID = get_data_ID()

    if len(data_ID) % batch_size == 0: # the total number of events
        flags = int(len(data_ID) / FLAGS.batch_size)
    else:
        flags = int(len(data_ID) / FLAGS.batch_size) + 1
    for i in range(flags):
        with torch.no_grad():
            x, x_len, y = get_df_batch(i, batch_size, tokenizer=tokenizer)
            x_emb = Word_ids2SeqStates(x, bert, 3, cuda=True) 
            batchsize, max_seq_len, max_sent_len,                                     emb_dim = x_emb.shape
            sent_feature = sentiModel( 
                x_emb.reshape(
                    [-1, max_sent_len, emb_dim]
                ) 
            ).reshape(
                [batchsize, max_seq_len, -1]
            )
            rdm_hiddens, rdm_outs = rdm_model(sent_feature, x_len, df_init_states)
        #         t_ssq = sess.run(rdm_train.out_seq, feed_dic)# t_ssq = [batchsize, max_seq, scores]
            print("batch %d"%i)
            if len(ssq) > 0:
                ssq.extend([rdm_classifier(h) for h in rdm_hiddens])
            else:
                ssq = [rdm_classifier(h) for h in rdm_hiddens]

    print(get_curtime() + " Now Start RL training ...")
    counter = 0
    sum_rw = 0.0 # sum of rewards

    data_len = get_data_len()

    while True:
        if counter > FLAGS.OBSERVE:
            sum_rw += np.mean(rw)
            if counter % 200 == 0:
                sum_rw = sum_rw / 2000
                print( get_curtime() + " Step: " + str(step) 
                       + " REWARD IS " + str(sum_rw) 
                     )
                logger.info( get_curtime() + 
                             " Step: " + str(step) + 
                            " REWARD IS " + str(sum_rw)
                           )
                if sum_rw > t_rw:
                    print("Retch The Target Reward")
                    logger.info("Retch The Target Reward")
                    break
                if counter > t_steps:
                    print("Retch The Target Steps")
                    logger.info("Retch The Target Steps")
                    break
                sum_rw = 0.0
            s_state, s_x, s_isStop, s_rw = get_RL_Train_batch(D, FLAGS)
            stopScore, isStop, rl_new_state = cm_model(rdm_model, sentiModel, s_x, s_state)
            out_action = (stopScore*s_isStop).sum(axis=1)
            rl_cost = torch.mean((s_rw - out_action)*(s_rw - out_action))
            rl_cost.backward()
            rl_optim.step()

        input_x, input_y, ids, seq_states, max_id = get_rl_batch(ids, seq_states, isStop, max_id, 0, FLAGS, tokenizer=tokenizer)
        with torch.no_grad():
            x_emb = layer2seq(bert, input_x, cuda=True)
            batchsize, max_sent_len, emb_dim = x_emb.shape
            mss, isStop, mNewState = cm_model(rdm_model, sentiModel, x_emb, state)

        for j in range(FLAGS.batch_size):
            if random.random() < FLAGS.random_rate:
    #             isStop[j] = np.argmax(np.random.rand(2))
                isStop[j] = int(torch.rand(2).argmax())
            if seq_states[j] == data_len[ids[j]]:
                isStop[j] = 1

        # eval
        rw = get_reward(isStop, mss, ssq, ids, seq_states)

        for j in range(FLAGS.batch_size):
            D.append((state[0][j], input_x[j], isStop[j], rw[j]))
            if len(D) > FLAGS.max_memory:
                D.popleft()

        state = mNewState
        for j in range(FLAGS.batch_size):
            if isStop[j] == 1:
                # init_states = np.zeros([FLAGS.batch_size, FLAGS.hidden_dim], dtype=np.float32)
                # feed_dic = {rl_model.init_states: init_states}
                # state[j] = sess.run(rl_model.df_state, feed_dic)
    #             state[j] = np.zeros([FLAGS.hidden_dim], dtype=np.float32)
                state[0][j].fill_(0)
        counter += 1.0

## 联合学习部分


### 工具函数

In [4]:
def words2tensor(bert, layer, cuda=False):
    if cuda:
        outs = [bert( torch.tensor([input_]).cuda())
                for input_ in layer]   
    else: 
        outs = [bert( torch.tensor([input_]))
                    for input_ in layer]
    states = [item[0][0] for item in outs]
    return rnn_utils.pad_sequence(states, batch_first=True)


> normalized the data for unbalanced learning:
```python
weights = WeightsForUmbalanced(emoReader.label)
weights
```

In [5]:
def WeightsForUmbalanced(data_label):
    _, _, labels = data_label.shape
    label_cnt = data_label.reshape([-1, labels]).sum(axis=0)
    weights = 1.0/label_cnt
    normalized_weights = weights/sum(weights)
    return normalized_weights

### 求各个任务的损失值

In [6]:
def senti_loss( xst, yst, lst, 
                bert, transformer, task_emb,
                senti_classifier, senti_loss_fn,
                cuda=False
               ):
    tensors = words2tensor(bert, xst, cuda) + task_emb
    y_label = torch.tensor(yst.argmax(axis=1)).cuda() if cuda else torch.tensor(yst.argmax(axis=1))
    senti_feature = transformer(tensors.transpose(0, 1)).transpose(0, 1)
    cls_feature = senti_feature.max(axis=1)[0]
    senti_scores = senti_classifier(cls_feature)
    loss = senti_loss_fn(senti_scores, y_label)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, senti_scores.argmax(axis=1))
    return loss, acc

In [7]:
def subj_loss(  xsj, ysj, lsj,
                bert, transformer, task_emb,
                subj_classifier, subj_loss_fn,
                cuda=False
               ):
    tensors = words2tensor(bert, xsj, cuda) + task_emb
    y_label = torch.tensor(ysj.argmax(axis=1)) if not cuda else torch.tensor(ysj.argmax(axis=1)).cuda()
    subj_feature = transformer(tensors.transpose(0, 1)).transpose(0, 1)
    cls_feature = subj_feature.max(axis=1)[0]
    subj_scores = subj_classifier(cls_feature)
    loss = subj_loss_fn(subj_scores, y_label)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, subj_scores.argmax(axis=1))
    return loss, acc

In [8]:
def emotion_loss(   xem, yem, lem,
                    bert, transformer, task_emb,
                    emotion_classifier, emo_loss_fn, 
                    cuda
                ):
    tensors = words2tensor(bert, xem, cuda) + task_emb
    y_label = torch.tensor(yem.argmax(axis=1)) if not cuda else torch.tensor(yem.argmax(axis=1)).cuda()
    emo_feature = transformer(tensors.transpose(0, 1)).transpose(0, 1)
    cls_feature = emo_feature.max(axis=1)[0]
    emo_scores = emotion_classifier(cls_feature)
    loss = emo_loss_fn(emo_scores, y_label)
    acc, _, _, _, _, _, _ = Count_Accs(y_label, emo_scores.argmax(axis=1))
    return loss, acc

### 训练各个任务

In [9]:
def emo_cls_train(emotion_reader, bert, transformer, 
                  task_embedding, emotion_classifier,
                  train_epochs, emo_loss_fn,
                  cuda=False
                 ):
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': emotion_classifier.parameters(), 'lr': 1e-6}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    
    task_emb = task_embedding(torch.tensor([2]).cuda()) if cuda else task_embedding(torch.tensor([2]))
    
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for x, y, l in emotion_reader.iter():
            emo_loss, emo_acc = emotion_loss(
                                            x, y, l,
                                            bert, transformer, task_emb,
                                            emotion_classifier, emo_loss_fn, 
                                            cuda
                                   )
            emo_loss.backward(retain_graph=True)
            optim.step()
            losses[int(step%10)] = emo_loss.cpu()
            accs[int(step%10)] = emo_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, emo_loss, emo_acc))
            if step %10 == 9:
                print('emotion task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )       
            step += 1 

In [10]:
def subj_cls_train(subj_reader, bert, transformer, task_embedding,
                   subj_classifier, train_epochs, subj_loss_fn,
                   cuda=False
                  ):
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':5e-5},
                                {'params': transformer.parameters(), 'lr': 5e-5},
                                {'params': subj_classifier.parameters(), 'lr': 5e-5}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    task_emb = task_embedding(torch.tensor([1]).cuda()) if cuda else task_embedding(torch.tensor([1]))
    
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for x, y, l in subj_reader.iter():
            sj_loss, sj_acc = subj_loss(x, y, l,
                                    bert, transformer, task_emb,
                                    subj_classifier, subj_loss_fn, 
                                    cuda
                                   )
            sj_loss.backward(retain_graph=True)
            optim.step()
            losses[int(step%10)] = sj_loss.cpu()
            accs[int(step%10)] = sj_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, sj_loss, sj_acc))
            if step %10 == 9:
                print('subjective task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )       
            step += 1 

In [11]:
def senti_cls_train(senti_reader, bert, transformer,
                    task_embedding, senti_classifier,
                    train_epochs, senti_loss_fn,
                    cuda=False
                   ):
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':1e-6},
                                {'params': task_embedding.parameters(), 'lr':1e-5},
                                {'params': transformer.parameters(), 'lr': 1e-5},
                                {'params': senti_classifier.parameters(), 'lr': 1e-5}
                             ]
    )
    losses = np.zeros([10]) 
    accs = np.zeros([10])
    
    task_emb = task_embedding(torch.tensor([0]).cuda()) if cuda else task_embedding(torch.tensor([0]))
    
    optim.zero_grad()
    for epoch in range(train_epochs):
        step = 0
        for x, y, l in senti_reader.iter():
            st_loss, st_acc = senti_loss(x, y, l, 
                                    bert, transformer, task_emb,
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
            st_loss.backward(retain_graph=True)
            optim.step()
            losses[int(step%10)] = st_loss.cpu()
            accs[int(step%10)] = st_acc
            print("step:%d | loss/acc = %.3f/%.3f"%(step, st_loss, st_acc))
            if step %10 == 9:
                print('sentiment task: %6d: [%5d/%5d], senti_loss/senti_acc = %6.8f/%6.7f ' % ( step,
                                                                                epoch, train_epochs,
                                                                                losses.mean(), accs.mean(),
                                                                            )
                         )    
            step += 1 

### 联合训练函数

#### 三人舞
情感任务，主观客观性任务，　情绪分析任务，　三个共同训练的函数

In [12]:
def JointLearning(senti_reader, subj_reader, emotion_reader, 
                  bert, transformer, task_embedding, 
                  senti_classifier, subj_classifier, emotion_classifier,
                  cuda=False
                 ):
    
    #stage 3: deploy the trainning on the emotion classification task
#     emo_weights = torch.tensor(
#             WeightsForUmbalanced(
#                 emotion_reader.label
#             ),
#             dtype = torch.float32
#     )
#     emo_loss_fn = nn.CrossEntropyLoss(weight=emo_weights) if not cuda else nn.CrossEntropyLoss(weight=emo_weights.cuda())
#     emo_cls_train(
#         emotion_reader, 
#         bert, 
#         transformer, 
#         task_embedding, 
#         emotion_classifier, 
#         2, 
#         emo_loss_fn,
#         cuda
#     )
#     emotion_reader.reset_batchsize(5)    
    
    # stage 1: deploy the trainning on the senti classification task
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_reader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
    senti_cls_train(
                    senti_reader, 
                    bert, 
                    transformer, 
                    task_embedding, 
                    senti_classifier, 
                    2, 
                    senti_loss_fn,
                    cuda
                   )
    #stage 2: deploy the trainning on the subj classification task
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_reader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())
    subj_cls_train(
                   subj_reader, 
                   bert, 
                   transformer, 
                   task_embedding, 
                   subj_classifier, 
                   2, 
                   subj_loss_fn,
                   cuda
                  )
    
    joint_model_save_as = '/home/hadoop/ERD/MTLTrain/PreTrModel_epoch.pkl'
    torch.save(
        {
            "bert":bert.state_dict(),
            "transformer":transformer.state_dict(),
            "task_embedding":task_embedding.state_dict(),
            "senti_classifier": senti_classifier.state_dict(),
            "subj_classifier": subj_classifier.state_dict(),
#             "emotion_classifier": emotion_classifier.state_dict()
        },
        joint_model_save_as
    )
    print("saved pretrained model!")

    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-7},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': senti_classifier.parameters(), 'lr': 1e-6},
                                {'params': subj_classifier.parameters(), 'lr':1e-6},
                                {'params': emotion_classifier.parameters(), 'lr':1e-6}
                             ]
    )
    
    max_epoch = 100
    
    losses = np.zeros([3, 10]) 
    accs = np.zeros([3, 10])
    # [[senti_loss_1, ..., senti_loss_10], [subj_loss_1, ..., subj_loss_10], [emo_loss_1, ..., emo_loss_10]] 
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    subj_task_id = torch.tensor([1]) if not cuda else torch.tensor([1]).cuda()
#     emo_task_id = torch.tensor([2]) if not cuda else torch.tensor([2]).cuda()
    
    loss_weight = torch.tensor([0.333, 0.333, 0.333]) if not cuda else torch.tensor([0.333, 0.333, 0.333]).cuda()
    
    batchs = min(senti_reader.label.shape[0], subj_reader.label.shape[0], emotion_reader.label.shape[0])
    optim.zero_grad()
    for epoch in range(max_epoch):
        step = 0
        for ((xst, yst, lst), (xsj, ysj, lsj), (xem, yem, lem)) in zip(senti_reader.iter(), subj_reader.iter(), emotion_reader.iter()):
            st_loss, st_acc = senti_loss(xst, yst, lst, 
                                    bert, transformer, task_embedding(senti_task_id),
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
#             st_loss.backward()
            MTL_Loss = st_loss*loss_weight[0]
    
            losses[0][step%10] = st_loss.tolist()
            accs[0][step%10] = st_acc
             
            sj_loss, sj_acc = subj_loss(xsj, ysj, lsj,
                                    bert, transformer, task_embedding(subj_task_id),
                                    subj_classifier, subj_loss_fn,
                                    cuda
                                   )
#             sj_loss.backward()
            MTL_Loss += sj_loss*loss_weight[1]

            losses[1][step%10] = sj_loss.tolist()
            accs[1][step%10] = sj_acc
            
            emo_loss, emo_acc = emotion_loss(xem, yem, lem,
                                    bert, transformer, task_embedding(emo_task_id),
                                    emotion_classifier, emo_loss_fn,
                                    cuda
                                   )
#             emo_loss.backward()
            MTL_Loss += emo_loss*loss_weight[2]
    
            losses[2][step%10] = emo_loss.tolist()
            accs[2][step%10] = emo_acc
            
            MTL_Loss.backward()
            optim.step()
            optim.zero_grad()
            print("%6d|MTL_Loss:%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f | emo_loss/emo_acc = %6.8f/%6.7f" % (
                                                                                                step, MTL_Loss,        
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean(),
                                                                                                losses[2].mean(), accs[2].mean()
            )
            )
            if step % 10 == 9:
                print('%6d: [%5d/%5d], MTL_Loss|%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f | emo_loss/emo_acc = %6.8f/%6.7f' % (
                                                                                                step,
                                                                                                epoch,max_epoch, MTL_Loss,
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean(),
                                                                                                losses[2].mean(), accs[2].mean()
                                                                                            )
                     )
                loss_weight = torch.tensor(
                                            (1.0/accs.mean(axis=1))/sum(1.0/accs.mean(axis=1)),
                                            dtype=torch.float32
                                )
                print("\n\n loss_weight:", loss_weight)
            step += 1
        joint_model_save_as = '/home/hadoop/ERD/MTLTrain/jointModel_epoch%03d.pkl'% (epoch)
        torch.save(
            {
                "bert":bert.state_dict(),
                "transformer":transformer.state_dict(),
                "task_embedding":task_embedding.state_dict(),
                "senti_classifier": senti_classifier.state_dict(),
                "subj_classifier": subj_classifier.state_dict(),
                "emotion_classifier": emotion_classifier.state_dict()
            },
            joint_model_save_as
        )

#### 双人舞
情感分析和主观客观性任务联合训练的函数

In [13]:
def SentiSubjLearning(senti_reader, subj_reader, 
                  bert, transformer, task_embedding, 
                  senti_classifier, subj_classifier,
                  cuda=False
                 ):  
    
    # stage 1: deploy the trainning on the senti classification task
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_reader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
    #stage 2: deploy the trainning on the subj classification task
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_reader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())

    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-7},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': senti_classifier.parameters(), 'lr': 1e-6},
                                {'params': subj_classifier.parameters(), 'lr':1e-6}
                             ]
    )
    
    max_epoch = 10
    
    losses = np.zeros([3, 10]) 
    accs = np.zeros([3, 10])
    # [[senti_loss_1, ..., senti_loss_10], [subj_loss_1, ..., subj_loss_10], [emo_loss_1, ..., emo_loss_10]] 
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    subj_task_id = torch.tensor([1]) if not cuda else torch.tensor([1]).cuda()
    
    loss_weight = torch.tensor([0.5, 0.5]) if not cuda else torch.tensor([0.5, 0.5]).cuda()
    
    batchs = min(senti_reader.label.shape[0], subj_reader.label.shape[0])
    optim.zero_grad()
    for epoch in range(10, max_epoch+10):
        step = 0
        for ((xst, yst, lst), (xsj, ysj, lsj)) in zip(senti_reader.iter(), subj_reader.iter() ):
            st_loss, st_acc = senti_loss(xst, yst, lst, 
                                    bert, transformer, task_embedding(senti_task_id),
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
            MTL_Loss = st_loss*loss_weight[0]
            losses[0][step%10] = st_loss.tolist()
            accs[0][step%10] = st_acc
             
            sj_loss, sj_acc = subj_loss(xsj, ysj, lsj,
                                    bert, transformer, task_embedding(subj_task_id),
                                    subj_classifier, subj_loss_fn,
                                    cuda
                                   )
            MTL_Loss += sj_loss*loss_weight[1]
            losses[1][step%10] = sj_loss.tolist()
            accs[1][step%10] = sj_acc
            
            MTL_Loss.backward()
            optim.step()
            optim.zero_grad()
            print("%6d %6d|MTL_Loss:%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f " % (
                                                                                                step, batchs, MTL_Loss,        
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean()
            )
            )
            if step % 10 == 9:
                print('%6d %6d: [%5d/%5d], MTL_Loss|%6.8f, senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f' % (
                                                                                                step, batchs, 
                                                                                                epoch,max_epoch, MTL_Loss,
                                                                                                losses[0].mean(), accs[0].mean(),
                                                                                                losses[1].mean(), accs[1].mean()
                                                                                            )
                     )
                loss_weight = torch.tensor(
                                            (1.0/( accs.mean(axis=1)[:2] ) )/sum( 1.0/accs.mean(axis=1)[:2] ),
                                            dtype=torch.float32
                                )
                print("\n\n loss_weight:", loss_weight)
            step += 1
        joint_model_save_as = '/home/hadoop/ERD/MTLTrain/jointModel_epoch%03d.pkl'% (epoch)
        torch.save(
            {
                "bert":bert.state_dict(),
                "transformer":transformer.state_dict(),
                "task_embedding":task_embedding.state_dict(),
                "senti_classifier": senti_classifier.state_dict(),
                "subj_classifier": subj_classifier.state_dict()
            },
            joint_model_save_as
        )

#### 双任务增强
subj任务和sentiment任务来增强谣言检测模块

> 此时应当要能从Reader中随机地挑出一个batch出来训练

In [14]:
def MTLTrainRDMModel(rdm_model, bert, rdm_classifier,
                     transformer, task_embedding, senti_classifier, subj_classifier, 
                     sentiReader, subjReader, 
                    tokenizer, t_steps, new_data_len=[], logger=None, cuda=False, 
                        log_dir="RDMBertTrain"):
    batch_size = 10 
    max_gpu_batch = 2 #cannot load a larger batch into the limited memory, but we could  accumulates grads
    sentiReader.reset_batchsize(max_gpu_batch)
    subjReader.reset_batchsize(max_gpu_batch)
    
    assert(batch_size%max_gpu_batch == 0)
    
    sum_loss = np.zeros(4)
    sum_acc = np.zeros(3)
    
    t_acc = 0.9
    ret_acc = 0.0
    init_states = torch.zeros([1, 5, rdm_model.hidden_dim], dtype=torch.float32).cuda()
    
    senti_task_id = torch.tensor([0]) if not cuda else torch.tensor([0]).cuda()
    subj_task_id = torch.tensor([1]) if not cuda else torch.tensor([1]).cuda()
    
    weight = torch.tensor([2.0, 1.0], dtype=torch.float32).cuda()
    loss_fn = nn.CrossEntropyLoss(weight=weight)
    
    senti_weights = torch.tensor(
            WeightsForUmbalanced(
                sentiReader.label
            ),
            dtype=torch.float32
    )
    senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda()) if cuda else nn.CrossEntropyLoss(weight=senti_weights)
    
    subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subjReader.label
            ),
            dtype = torch.float32
    )
    subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights) if not cuda else nn.CrossEntropyLoss(weight=subj_weights.cuda())

    loss_weight = torch.tensor([0.8, 0.1, 0.1]) if not cuda else torch.tensor([0.8, 0.1, 0.1]).cuda()   
    optim = torch.optim.Adagrad([
                                {'params': bert.parameters(), 'lr':5e-7},
                                {'params': rdm_classifier.parameters(), 'lr': 5e-5},
                                {'params': rdm_model.parameters(), 'lr': 5e-5},
                                {'params': task_embedding.parameters(), 'lr':1e-6},
                                {'params': transformer.parameters(), 'lr': 1e-6},
                                {'params': senti_classifier.parameters(), 'lr': 1e-6},
                                {'params': subj_classifier.parameters(), 'lr':1e-6}
                             ]
    )
    
    writer = SummaryWriter(log_dir)
    
    acc_tmp = np.zeros([3, int(batch_size/max_gpu_batch)])
    loss_tmp = np.zeros([4, int(batch_size/max_gpu_batch)])
    
    for step in range(t_steps):
        optim.zero_grad()
        for j in range(int(batch_size/max_gpu_batch)):
            if len(new_data_len) > 0:
                x, x_len, y = get_df_batch(step, max_gpu_batch, new_data_len, tokenizer=tokenizer)
            else:
                x, x_len, y = get_df_batch(step, max_gpu_batch, tokenizer=tokenizer) 
            loss, acc = rdm_loss(x, y, x_len, bert, rdm_model, rdm_classifier, loss_fn)
            MTL_Loss = loss*loss_weight[0]
            
            xst, yst, lst = sentiReader.sample()
            st_loss, st_acc = senti_loss(xst, yst, lst, 
                                    bert, transformer, task_embedding(senti_task_id),
                                    senti_classifier, senti_loss_fn,
                                    cuda
                                   )
            MTL_Loss += st_loss*loss_weight[1]
            
            xsj, ysj, lsj = subjReader.sample()
            sj_loss, sj_acc = subj_loss(xsj, ysj, lsj,
                                    bert, transformer, task_embedding(subj_task_id),
                                    subj_classifier, subj_loss_fn,
                                    cuda
                                   )
            MTL_Loss += sj_loss*loss_weight[2]
            
            MTL_Loss.backward()
            
            loss_tmp[:, j] = np.array([MTL_Loss, loss, st_loss, sj_loss])
            acc_tmp[:, j] = np.array([acc, st_acc, sj_acc])
            
        optim.step()        
        writer.add_scalar('Train Loss', loss_tmp[0].mean(), step)
        writer.add_scalar('Train Accuracy', acc_tmp[0].mean(), step)
        
        sum_acc += acc_tmp.mean(axis=1)
        sum_loss += loss_tmp.mean(axis=1)
        
        print("%6d %6d|MTL_Loss:%6.8f, rdm_loss/rdm_acc = %6.8f/%6.7f | senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f " % (
                                                                                                step, t_steps, loss_tmp[0].mean(),        
                                                                                            loss_tmp[1].mean(), acc_tmp[0].mean(),
                                                                                            loss_tmp[2].mean(), acc_tmp[1].mean(),
                                                                                            loss_tmp[3].mean(), acc_tmp[2].mean()
            )
            )
        
        if step % 10 == 9:
            sum_loss = sum_loss / 10
            sum_acc = sum_acc / 10
            print("MTL_Loss:%6.8f, rdm_loss/rdm_acc = %6.8f/%6.7f | senti_loss/senti_acc = %6.8f/%6.7f | subj_loss/subj_acc = %6.8f/%6.7f " % (
                                                                                            sum_loss[0],        
                                                                                            sum_loss[1], sum_acc[0],
                                                                                            sum_loss[2], sum_acc[1],
                                                                                            sum_loss[3], sum_acc[2]
            )
            )
            if step%100 == 99:
                rdm_save_as = '/home/hadoop/ERD/%s/rdmModel_epoch%03d.pkl'% (log_dir, step/100, sum_acc)
                torch.save(
                    {
                        "bert":bert.state_dict(),
                        "transformer":transformer.state_dict(),
                        "task_embedding":task_embedding.state_dict(),
                        "senti_classifier": senti_classifier.state_dict(),
                        "subj_classifier": subj_classifier.state_dict(),
                        "rmdModel":rdm_model.state_dict(),
                        "rdm_classifier": rdm_classifierdm.state_dict()
                    },
                    rdm_save_as
                )
#                 rdm_model, bert, sentiModel, rdm_classifier
            sum_acc = 0.0
            sum_loss = 0.0

    print(get_curtime() + " Train df Model End.")
    logger.info(get_curtime() + " Train df Model End.")
    return ret_acc

### 主函数部分

#### 模型部分

In [3]:
tt = BertTokenizer.from_pretrained("bert-base-uncased", cached_dir = "/home/hadoop/transformer_pretrained_models/bert-base-uncased-pytorch_model.bin")
bb = BertModel.from_pretrained("bert-base-uncased")

```
bb.save_pretrained('./bertModel/')  # save
tt.save_pretrained('./bertModel/')  # save
```

In [16]:
task_embedding = nn.Embedding(3, 768)

encoder_layer = nn.TransformerEncoderLayer(768, 8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, 1)

subj_cls = nn.Linear(768, 2)

bert = bb.cuda()
transformer = transformer_encoder.cuda()
task_embedding = task_embedding.cuda()
subj_cls = subj_cls.cuda()

senti_cls = nn.Linear(768, 2).cuda()

#### 各个任务的数据

In [17]:
subj_file = "/home/hadoop/rotten_imdb/subj.data"
obj_file = "/home/hadoop/rotten_imdb/obj.data"
tr, dev, te = load_data(subj_file, obj_file)

subj_train_reader = SubjObjReader(tr, 20, tt)
subj_valid_reader = SubjObjReader(dev, 20, tt)
subj_test_reader =  SubjObjReader(te, 20, tt)

In [18]:
train_file = "/home/hadoop/trainingandtestdata/training.1600000.processed.noemoticon.csv"
test_file = "/home/hadoop/trainingandtestdata/testdata.manual.2009.06.14.csv"
train_set, test_set = tsentiLoader.load_data(train_file, test_file)

/home/hadoop/trainingandtestdata/training.1600000.processed.noemoticon.csv
/home/hadoop/trainingandtestdata/testdata.manual.2009.06.14.csv


In [19]:
senti_train_reader = tsentiLoader.tSentiReader(train_set[:10000], 20, tt)
senti_train_reader.label = np.delete(senti_train_reader.label, 1, axis=2)
# senti_valid_reader = tsentiLoader.tSentiReader(train_set[10000:10100], 20, tt)
# senti_test_reader =  tsentiLoader.tSentiReader(test_set, 20, tt)

In [20]:
load_data_fast()

max_sent: 187 ,  max_seq_len: 101
5802 data loaded


### 调试各个任务的部分

#### 调试subjective的任务

In [ ]:
subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_train_reader.label
            ),
            dtype = torch.float32
    ).cuda()
subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights)
subj_cls_train(subj_train_reader, bert, transformer, task_embedding, subj_cls, 1, subj_loss_fn, cuda=True)

In [ ]:
joint_model_save_as = '/home/hadoop/ERD/MTLTrain/SubjAfterSentiModel_epoch.pkl'
torch.save(
    {
        "bert":bert.state_dict(),
        "transformer":transformer.state_dict(),
        "task_embedding":task_embedding.state_dict(),
#         "senti_classifier": senti_cls.state_dict(),
        "subj_classifier": subj_cls.state_dict(),
#         "emotion_classifier": emotion_classifier.state_dict()
    },
    joint_model_save_as
)

> __test code__
``` python
for xsj, ysj, lsj in subj_train_reader.iter():
    loss = subj_loss(  xsj, ysj, lsj,
                bb, transformer_encoder, task_embedding(torch.tensor([1])),
                subj_cls, subj_loss_fn
               )
    break
```
> __train stage__
```python
subj_weights = torch.tensor(
            WeightsForUmbalanced(
                subj_train_reader.label
            ),
            dtype = torch.float32
    ).cuda()
subj_loss_fn = nn.CrossEntropyLoss(weight=subj_weights)
subj_cls_train(subj_train_reader, bert, transformer, task_embedding, subj_cls, 1, subj_loss_fn, cuda=True)
```
_subj task 的训练，在当前的参数配置下，一个ｅｐｏｃｈ可以调到很好的状态_

#### 调试sentiment 的任务

In [ ]:
senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_train_reader.label
            ),
            dtype=torch.float32
    )
senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda())
senti_cls_train(senti_train_reader, bert, transformer,
                    task_embedding, senti_cls,
                    1, senti_loss_fn,
                    cuda=True
                   )

> __测试sentiment analysis的task__
``` python
senti_weights = torch.tensor(
            WeightsForUmbalanced(
                senti_train_reader.label
            ),
            dtype=torch.float32
    )
senti_loss_fn = nn.CrossEntropyLoss(weight=senti_weights.cuda())
senti_cls_train(senti_train_reader, bert, transformer,
                    task_embedding, senti_cls,
                    2, senti_loss_fn,
                    cuda=True
                   )
``` 
_这个任务至少需要５个ｅｐｏｃｈ才能训练出来个大概的样子_

In [ ]:
senti_cls_train(senti_train_reader, bert, transformer,
                    task_embedding, senti_cls,
                    1, senti_loss_fn,
                    cuda=True
                   )

In [ ]:
joint_model_save_as = '/home/hadoop/ERD/MTLTrain/SentiPretrainModel_epoch.pkl'
torch.save(
    {
        "bert":bert.state_dict(),
        "transformer":transformer.state_dict(),
        "task_embedding":task_embedding.state_dict(),
        "senti_classifier": senti_cls.state_dict(),
#         "subj_classifier": subj_classifier.state_dict(),
#         "emotion_classifier": emotion_classifier.state_dict()
    },
    joint_model_save_as
)

#### 调试emotion classification　任务

In [23]:
file2 = "/home/hadoop/EmoNet-PyTorch/twitter30.json"

In [24]:
sentences, labels, emotion_set = load_data_from_file(file2)

In [25]:
emoReader = EmotionReader(sentences, labels, 20, tt)

In [26]:
emo_cls = nn.Linear(768, 6).cuda()

In [27]:
emo_weights = torch.tensor(
            WeightsForUmbalanced(
                emoReader.label
            ),
            dtype = torch.float32
    )
emo_loss_fn = nn.CrossEntropyLoss(weight=emo_weights.cuda())

In [28]:
emo_cls_train(
        emoReader, 
        bert, 
        transformer, 
        task_embedding, 
        emo_cls, 
        10, 
        emo_loss_fn, 
        cuda=True
    )

step:0 | loss/acc = 1.882/0.200
step:1 | loss/acc = 1.654/0.200
step:2 | loss/acc = 1.572/0.300
step:3 | loss/acc = 1.824/0.100
step:4 | loss/acc = 1.623/0.300
step:5 | loss/acc = 1.653/0.100
step:6 | loss/acc = 1.976/0.200
step:7 | loss/acc = 2.144/0.100
step:8 | loss/acc = 1.669/0.200
step:9 | loss/acc = 1.820/0.200
emotion task:      9: [    0/   10], senti_loss/senti_acc = 1.78169487/0.1900000 
step:10 | loss/acc = 2.021/0.100
step:11 | loss/acc = 1.685/0.250
step:12 | loss/acc = 1.759/0.100
step:13 | loss/acc = 1.996/0.050
step:14 | loss/acc = 1.896/0.100
step:15 | loss/acc = 1.873/0.150
step:16 | loss/acc = 1.850/0.100
step:17 | loss/acc = 1.942/0.100
step:18 | loss/acc = 2.001/0.100
step:19 | loss/acc = 1.597/0.100
emotion task:     19: [    0/   10], senti_loss/senti_acc = 1.86189852/0.1150000 
step:20 | loss/acc = 2.058/0.200
step:21 | loss/acc = 1.708/0.050
step:22 | loss/acc = 1.761/0.150
step:23 | loss/acc = 1.834/0.200
step:24 | loss/acc = 2.066/0.050
step:25 | loss/acc = 

step:199 | loss/acc = 1.826/0.400
emotion task:    199: [    0/   10], senti_loss/senti_acc = 1.85852419/0.2550000 
step:200 | loss/acc = 1.728/0.250
step:201 | loss/acc = 1.672/0.400
step:202 | loss/acc = 1.931/0.450
step:203 | loss/acc = 1.873/0.100
step:204 | loss/acc = 1.673/0.150
step:205 | loss/acc = 1.859/0.400
step:206 | loss/acc = 1.718/0.400
step:207 | loss/acc = 1.692/0.250
step:208 | loss/acc = 1.761/0.150
step:209 | loss/acc = 1.872/0.350
emotion task:    209: [    0/   10], senti_loss/senti_acc = 1.77790780/0.2900000 
step:210 | loss/acc = 1.753/0.150
step:211 | loss/acc = 1.928/0.400
step:212 | loss/acc = 1.686/0.450
step:213 | loss/acc = 1.837/0.250
step:214 | loss/acc = 1.702/0.350
step:215 | loss/acc = 1.878/0.350
step:216 | loss/acc = 1.718/0.150
step:217 | loss/acc = 1.846/0.350
step:218 | loss/acc = 1.843/0.150
step:219 | loss/acc = 1.829/0.300
emotion task:    219: [    0/   10], senti_loss/senti_acc = 1.80200164/0.2900000 
step:220 | loss/acc = 1.767/0.300
step:2

step:392 | loss/acc = 1.780/0.300
step:393 | loss/acc = 1.812/0.100
step:394 | loss/acc = 1.746/0.300
step:395 | loss/acc = 1.869/0.200
step:396 | loss/acc = 1.780/0.450
step:397 | loss/acc = 1.804/0.250
step:398 | loss/acc = 1.811/0.150
step:399 | loss/acc = 1.722/0.350
emotion task:    399: [    0/   10], senti_loss/senti_acc = 1.77540942/0.2800000 
step:400 | loss/acc = 1.802/0.100
step:401 | loss/acc = 1.788/0.250
step:402 | loss/acc = 1.734/0.250
step:403 | loss/acc = 1.866/0.350
step:404 | loss/acc = 1.806/0.150
step:405 | loss/acc = 1.758/0.300
step:406 | loss/acc = 1.838/0.150
step:407 | loss/acc = 1.758/0.450
step:408 | loss/acc = 1.797/0.300
step:409 | loss/acc = 1.689/0.400
emotion task:    409: [    0/   10], senti_loss/senti_acc = 1.78367494/0.2700000 
step:410 | loss/acc = 1.755/0.350
step:411 | loss/acc = 1.702/0.500
step:412 | loss/acc = 1.735/0.150
step:413 | loss/acc = 1.778/0.250
step:414 | loss/acc = 1.669/0.350
step:415 | loss/acc = 1.707/0.350
step:416 | loss/acc 

step:588 | loss/acc = 1.790/0.200
step:589 | loss/acc = 1.765/0.100
emotion task:    589: [    0/   10], senti_loss/senti_acc = 1.76209234/0.2500000 
step:590 | loss/acc = 1.822/0.200
step:591 | loss/acc = 1.737/0.400
step:592 | loss/acc = 1.848/0.200
step:593 | loss/acc = 1.835/0.150
step:594 | loss/acc = 1.817/0.100
step:595 | loss/acc = 1.738/0.250
step:596 | loss/acc = 1.788/0.300
step:597 | loss/acc = 1.740/0.300
step:598 | loss/acc = 1.792/0.250
step:599 | loss/acc = 1.733/0.200
emotion task:    599: [    0/   10], senti_loss/senti_acc = 1.78505458/0.2350000 
step:600 | loss/acc = 1.740/0.200
step:601 | loss/acc = 1.730/0.350
step:602 | loss/acc = 1.797/0.150
step:603 | loss/acc = 1.754/0.250
step:604 | loss/acc = 1.771/0.250
step:605 | loss/acc = 1.842/0.150
step:606 | loss/acc = 1.714/0.250
step:607 | loss/acc = 1.734/0.200
step:608 | loss/acc = 1.756/0.150
step:609 | loss/acc = 1.740/0.150
emotion task:    609: [    0/   10], senti_loss/senti_acc = 1.75792181/0.2100000 
step:6

step:781 | loss/acc = 1.696/0.300
step:782 | loss/acc = 1.793/0.200
step:783 | loss/acc = 1.753/0.250
step:784 | loss/acc = 1.769/0.200
step:785 | loss/acc = 1.779/0.050
step:786 | loss/acc = 1.635/0.250
step:787 | loss/acc = 1.790/0.050
step:788 | loss/acc = 1.724/0.150
step:789 | loss/acc = 1.757/0.200
emotion task:    789: [    0/   10], senti_loss/senti_acc = 1.73086311/0.2000000 
step:790 | loss/acc = 1.787/0.200
step:791 | loss/acc = 1.701/0.200
step:792 | loss/acc = 1.675/0.450
step:793 | loss/acc = 1.632/0.400
step:794 | loss/acc = 1.705/0.250
step:795 | loss/acc = 1.779/0.250
step:796 | loss/acc = 1.660/0.500
step:797 | loss/acc = 1.721/0.200
step:798 | loss/acc = 1.713/0.250
step:799 | loss/acc = 1.802/0.150
emotion task:    799: [    0/   10], senti_loss/senti_acc = 1.71773971/0.2850000 
step:800 | loss/acc = 1.711/0.200
step:801 | loss/acc = 1.830/0.050
step:802 | loss/acc = 1.769/0.250
step:803 | loss/acc = 1.673/0.300
step:804 | loss/acc = 1.626/0.300
step:805 | loss/acc 

...

step:77 | loss/acc = 1.136/0.650
step:78 | loss/acc = 1.308/0.550
step:79 | loss/acc = 1.197/0.550
emotion task:     79: [    9/   10], senti_loss/senti_acc = 1.24595260/0.5250000 
step:80 | loss/acc = 1.349/0.600
step:81 | loss/acc = 1.398/0.400
step:82 | loss/acc = 1.274/0.500
step:83 | loss/acc = 1.332/0.550
step:84 | loss/acc = 1.320/0.500
step:85 | loss/acc = 1.353/0.650
step:86 | loss/acc = 1.106/0.500
step:87 | loss/acc = 1.087/0.700
step:88 | loss/acc = 1.226/0.600
step:89 | loss/acc = 1.245/0.650
emotion task:     89: [    9/   10], senti_loss/senti_acc = 1.26910301/0.5650000 
step:90 | loss/acc = 1.247/0.500
step:91 | loss/acc = 1.641/0.500
step:92 | loss/acc = 1.237/0.550
step:93 | loss/acc = 1.905/0.550
step:94 | loss/acc = 1.531/0.500
step:95 | loss/acc = 1.307/0.550
step:96 | loss/acc = 1.485/0.500
step:97 | loss/acc = 1.282/0.450
step:98 | loss/acc = 1.316/0.350
step:99 | loss/acc = 1.550/0.400
emotion task:     99: [    9/   10], senti_loss/senti_acc = 1.44994441/0.4850

step:271 | loss/acc = 1.185/0.600
step:272 | loss/acc = 1.267/0.550
step:273 | loss/acc = 1.197/0.550
step:274 | loss/acc = 1.044/0.750
step:275 | loss/acc = 1.433/0.450
step:276 | loss/acc = 1.425/0.550
step:277 | loss/acc = 1.034/0.750
step:278 | loss/acc = 1.427/0.350
step:279 | loss/acc = 1.362/0.600
emotion task:    279: [    9/   10], senti_loss/senti_acc = 1.28882244/0.5600000 
step:280 | loss/acc = 1.665/0.300
step:281 | loss/acc = 1.531/0.700
step:282 | loss/acc = 1.373/0.500
step:283 | loss/acc = 1.193/0.500
step:284 | loss/acc = 1.391/0.500
step:285 | loss/acc = 1.330/0.500
step:286 | loss/acc = 1.244/0.500
step:287 | loss/acc = 1.291/0.500
step:288 | loss/acc = 1.150/0.800
step:289 | loss/acc = 1.413/0.450
emotion task:    289: [    9/   10], senti_loss/senti_acc = 1.35798595/0.5250000 
step:290 | loss/acc = 1.748/0.200
step:291 | loss/acc = 1.050/0.650
step:292 | loss/acc = 1.233/0.550
step:293 | loss/acc = 1.292/0.600
step:294 | loss/acc = 1.368/0.550
step:295 | loss/acc 

step:467 | loss/acc = 1.500/0.350
step:468 | loss/acc = 1.500/0.500
step:469 | loss/acc = 1.692/0.500
emotion task:    469: [    9/   10], senti_loss/senti_acc = 1.36728178/0.4800000 
step:470 | loss/acc = 1.251/0.400
step:471 | loss/acc = 1.350/0.700
step:472 | loss/acc = 1.429/0.450
step:473 | loss/acc = 1.313/0.500
step:474 | loss/acc = 1.380/0.500
step:475 | loss/acc = 1.125/0.650
step:476 | loss/acc = 1.200/0.650
step:477 | loss/acc = 1.277/0.450
step:478 | loss/acc = 1.211/0.550
step:479 | loss/acc = 1.435/0.500
emotion task:    479: [    9/   10], senti_loss/senti_acc = 1.29707283/0.5350000 
step:480 | loss/acc = 1.515/0.550
step:481 | loss/acc = 1.410/0.450
step:482 | loss/acc = 1.177/0.600
step:483 | loss/acc = 0.895/0.700
step:484 | loss/acc = 1.639/0.300
step:485 | loss/acc = 1.400/0.600
step:486 | loss/acc = 1.060/0.750
step:487 | loss/acc = 1.310/0.550
step:488 | loss/acc = 1.072/0.700
step:489 | loss/acc = 1.270/0.500
emotion task:    489: [    9/   10], senti_loss/senti_

step:660 | loss/acc = 1.127/0.650
step:661 | loss/acc = 1.218/0.550
step:662 | loss/acc = 1.261/0.450
step:663 | loss/acc = 1.266/0.450
step:664 | loss/acc = 0.991/0.700
step:665 | loss/acc = 1.182/0.550
step:666 | loss/acc = 1.173/0.550
step:667 | loss/acc = 1.159/0.750
step:668 | loss/acc = 1.160/0.550
step:669 | loss/acc = 1.221/0.500
emotion task:    669: [    9/   10], senti_loss/senti_acc = 1.17577593/0.5700000 
step:670 | loss/acc = 1.397/0.500
step:671 | loss/acc = 1.325/0.500
step:672 | loss/acc = 1.029/0.700
step:673 | loss/acc = 1.131/0.450
step:674 | loss/acc = 1.383/0.300
step:675 | loss/acc = 0.953/0.600
step:676 | loss/acc = 1.085/0.750
step:677 | loss/acc = 1.368/0.400
step:678 | loss/acc = 1.048/0.700
step:679 | loss/acc = 1.312/0.550
emotion task:    679: [    9/   10], senti_loss/senti_acc = 1.20308720/0.5450000 
step:680 | loss/acc = 1.207/0.450
step:681 | loss/acc = 1.097/0.450
step:682 | loss/acc = 1.377/0.350
step:683 | loss/acc = 1.541/0.400
step:684 | loss/acc 

step:856 | loss/acc = 1.679/0.500
step:857 | loss/acc = 1.262/0.600
step:858 | loss/acc = 1.400/0.500
step:859 | loss/acc = 1.413/0.450
emotion task:    859: [    9/   10], senti_loss/senti_acc = 1.34471283/0.5600000 
step:860 | loss/acc = 1.312/0.500
step:861 | loss/acc = 1.006/0.650
step:862 | loss/acc = 1.537/0.450
step:863 | loss/acc = 1.338/0.450
step:864 | loss/acc = 1.188/0.450
step:865 | loss/acc = 1.313/0.350
step:866 | loss/acc = 1.072/0.600
step:867 | loss/acc = 1.270/0.600
step:868 | loss/acc = 1.507/0.500
step:869 | loss/acc = 1.334/0.550
emotion task:    869: [    9/   10], senti_loss/senti_acc = 1.28769192/0.5100000 
step:870 | loss/acc = 1.504/0.300
step:871 | loss/acc = 1.366/0.400
step:872 | loss/acc = 1.202/0.500
step:873 | loss/acc = 1.218/0.450
step:874 | loss/acc = 1.163/0.650
step:875 | loss/acc = 1.128/0.650
step:876 | loss/acc = 1.297/0.650
step:877 | loss/acc = 0.930/0.850
step:878 | loss/acc = 1.462/0.500
step:879 | loss/acc = 1.092/0.600
emotion task:    879

### 联合训练部分

#### 导入预训练模型

In [25]:
senti_save_as = '/home/hadoop/ERD/MTLTrain/SentiPretrainModel_epoch.pkl'

checkpoint = torch.load(senti_save_as)

senti_cls.load_state_dict(checkpoint['senti_classifier'])

<All keys matched successfully>

In [26]:
SentiSubj_saved_as =  '/home/hadoop/ERD/MTLTrain/SubjAfterSentiModel_epoch.pkl'
checkpoint = torch.load(SentiSubj_saved_as)
bert.load_state_dict(checkpoint['bert'])
transformer.load_state_dict(checkpoint['transformer'])
task_embedding.load_state_dict(checkpoint['task_embedding'])
subj_cls.load_state_dict(checkpoint['subj_classifier'])

<All keys matched successfully>

In [21]:
joint_save_as = '/home/hadoop/ERD/MTLTrain/jointModel_epoch015.pkl'
checkpoint = torch.load(joint_save_as)
senti_cls.load_state_dict(checkpoint['senti_classifier'])
bert.load_state_dict(checkpoint['bert'])
transformer.load_state_dict(checkpoint['transformer'])
task_embedding.load_state_dict(checkpoint['task_embedding'])
subj_cls.load_state_dict(checkpoint['subj_classifier'])

<All keys matched successfully>

#### 联合训练

In [28]:
SentiSubjLearning(senti_train_reader, subj_train_reader, 
                  bert, transformer, task_embedding, 
                  senti_cls, subj_cls,
                  cuda=True
                 )

     0    425|MTL_Loss:0.29332089, senti_loss/senti_acc = 0.05127532/0.0750000 | subj_loss/subj_acc = 0.00738885/0.1000000 
     1    425|MTL_Loss:0.19184102, senti_loss/senti_acc = 0.08910429/0.1500000 | subj_loss/subj_acc = 0.00792809/0.2000000 
     2    425|MTL_Loss:0.41711080, senti_loss/senti_acc = 0.15514933/0.2150000 | subj_loss/subj_acc = 0.02530521/0.2850000 
     3    425|MTL_Loss:0.18992978, senti_loss/senti_acc = 0.18254753/0.3150000 | subj_loss/subj_acc = 0.03589297/0.3800000 
     4    425|MTL_Loss:0.26974079, senti_loss/senti_acc = 0.22672767/0.4000000 | subj_loss/subj_acc = 0.04566098/0.4800000 
     5    425|MTL_Loss:0.22977003, senti_loss/senti_acc = 0.26549595/0.4800000 | subj_loss/subj_acc = 0.05284671/0.5750000 
     6    425|MTL_Loss:0.36548451, senti_loss/senti_acc = 0.32818922/0.5450000 | subj_loss/subj_acc = 0.06325034/0.6700000 
     7    425|MTL_Loss:0.31482914, senti_loss/senti_acc = 0.38309467/0.6200000 | subj_loss/subj_acc = 0.07131071/0.7650000 
     8  

KeyboardInterrupt: 

In [ ]:
JointLearning(senti_train_reader, subj_train_reader, emoReader, 
                  bert, transformer, task_embedding, 
                  senti_cls, subj_cls, emo_cls,
                  cuda=True
                 )

step:0 | loss/acc = 0.702/0.500
step:1 | loss/acc = 0.696/0.450
step:2 | loss/acc = 0.697/0.500
step:3 | loss/acc = 0.690/0.500
step:4 | loss/acc = 0.687/0.450
step:5 | loss/acc = 0.668/0.600
step:6 | loss/acc = 0.692/0.500
step:7 | loss/acc = 0.743/0.350
step:8 | loss/acc = 0.651/0.650
step:9 | loss/acc = 0.648/0.750
sentiment task:      9: [    0/    2], senti_loss/senti_acc = 0.68745612/0.5250000 
step:10 | loss/acc = 0.708/0.450
step:11 | loss/acc = 0.664/0.700
step:12 | loss/acc = 0.658/0.750
step:13 | loss/acc = 0.681/0.400
step:14 | loss/acc = 0.672/0.600
step:15 | loss/acc = 0.706/0.450
step:16 | loss/acc = 0.737/0.450
step:17 | loss/acc = 0.781/0.400
step:18 | loss/acc = 0.742/0.450
step:19 | loss/acc = 0.703/0.500
sentiment task:     19: [    0/    2], senti_loss/senti_acc = 0.70535051/0.5150000 
step:20 | loss/acc = 0.784/0.400
step:21 | loss/acc = 0.757/0.450
step:22 | loss/acc = 0.689/0.450
step:23 | loss/acc = 0.720/0.400
step:24 | loss/acc = 0.664/0.650
step:25 | loss/ac

step:198 | loss/acc = 0.631/0.800
step:199 | loss/acc = 0.658/0.550
sentiment task:    199: [    0/    2], senti_loss/senti_acc = 0.63688825/0.6850000 
step:200 | loss/acc = 0.669/0.600
step:201 | loss/acc = 0.604/0.750
step:202 | loss/acc = 0.628/0.750
step:203 | loss/acc = 0.574/0.700
step:204 | loss/acc = 0.590/0.700
step:205 | loss/acc = 0.649/0.600
step:206 | loss/acc = 0.694/0.450
step:207 | loss/acc = 0.741/0.350
step:208 | loss/acc = 0.583/0.750
step:209 | loss/acc = 0.693/0.500
sentiment task:    209: [    0/    2], senti_loss/senti_acc = 0.64253100/0.6150000 
step:210 | loss/acc = 0.694/0.500
step:211 | loss/acc = 0.580/0.750
step:212 | loss/acc = 0.652/0.650
step:213 | loss/acc = 0.713/0.500
step:214 | loss/acc = 0.623/0.700
step:215 | loss/acc = 0.646/0.650
step:216 | loss/acc = 0.656/0.550
step:217 | loss/acc = 0.611/0.800
step:218 | loss/acc = 0.587/0.750
step:219 | loss/acc = 0.642/0.750
sentiment task:    219: [    0/    2], senti_loss/senti_acc = 0.64036837/0.6600000 


step:390 | loss/acc = 0.598/0.750
step:391 | loss/acc = 0.631/0.600
step:392 | loss/acc = 0.542/0.800
step:393 | loss/acc = 0.530/0.850
step:394 | loss/acc = 0.581/0.650
step:395 | loss/acc = 0.700/0.550
step:396 | loss/acc = 0.587/0.650
step:397 | loss/acc = 0.513/0.700
step:398 | loss/acc = 0.500/0.950
step:399 | loss/acc = 0.460/0.850
sentiment task:    399: [    0/    2], senti_loss/senti_acc = 0.56433112/0.7350000 
step:400 | loss/acc = 0.535/0.800
step:401 | loss/acc = 0.685/0.650
step:402 | loss/acc = 0.519/0.850
step:403 | loss/acc = 0.617/0.600
step:404 | loss/acc = 0.568/0.750
step:405 | loss/acc = 0.519/0.800
step:406 | loss/acc = 0.526/0.800
step:407 | loss/acc = 0.635/0.700
step:408 | loss/acc = 0.602/0.750
step:409 | loss/acc = 0.560/0.750
sentiment task:    409: [    0/    2], senti_loss/senti_acc = 0.57669491/0.7450000 
step:410 | loss/acc = 0.550/0.800
step:411 | loss/acc = 0.567/0.750
step:412 | loss/acc = 0.425/0.950
step:413 | loss/acc = 0.654/0.550
step:414 | loss/

step:87 | loss/acc = 0.577/0.800
step:88 | loss/acc = 0.486/0.800
step:89 | loss/acc = 0.461/0.750
sentiment task:     89: [    1/    2], senti_loss/senti_acc = 0.50369355/0.7850000 
step:90 | loss/acc = 0.435/0.900
step:91 | loss/acc = 0.541/0.750
step:92 | loss/acc = 0.382/0.950
step:93 | loss/acc = 0.541/0.800
step:94 | loss/acc = 0.497/0.850
step:95 | loss/acc = 0.339/0.900
step:96 | loss/acc = 0.543/0.850
step:97 | loss/acc = 0.487/0.750
step:98 | loss/acc = 0.602/0.650
step:99 | loss/acc = 0.579/0.700
sentiment task:     99: [    1/    2], senti_loss/senti_acc = 0.49462225/0.8100000 
step:100 | loss/acc = 0.532/0.650
step:101 | loss/acc = 0.578/0.700
step:102 | loss/acc = 0.443/0.800
step:103 | loss/acc = 0.563/0.750
step:104 | loss/acc = 0.427/0.800
step:105 | loss/acc = 0.509/0.750
step:106 | loss/acc = 0.638/0.650
step:107 | loss/acc = 0.469/0.750
step:108 | loss/acc = 0.514/0.700
step:109 | loss/acc = 0.451/0.900
sentiment task:    109: [    1/    2], senti_loss/senti_acc = 0

step:280 | loss/acc = 0.385/0.850
step:281 | loss/acc = 0.461/0.750
step:282 | loss/acc = 0.591/0.600
step:283 | loss/acc = 0.672/0.650
step:284 | loss/acc = 0.457/0.850
step:285 | loss/acc = 0.438/0.700
step:286 | loss/acc = 0.277/0.950
step:287 | loss/acc = 0.541/0.750
step:288 | loss/acc = 0.629/0.650
step:289 | loss/acc = 0.381/0.900
sentiment task:    289: [    1/    2], senti_loss/senti_acc = 0.48315102/0.7650000 
step:290 | loss/acc = 0.478/0.800
step:291 | loss/acc = 0.533/0.800
step:292 | loss/acc = 0.385/0.850
step:293 | loss/acc = 0.513/0.750
step:294 | loss/acc = 0.352/0.850
step:295 | loss/acc = 0.633/0.650
step:296 | loss/acc = 0.531/0.800
step:297 | loss/acc = 0.239/0.900
step:298 | loss/acc = 0.531/0.700
step:299 | loss/acc = 0.526/0.750
sentiment task:    299: [    1/    2], senti_loss/senti_acc = 0.47205535/0.7850000 
step:300 | loss/acc = 0.450/0.750
step:301 | loss/acc = 0.520/0.700
step:302 | loss/acc = 0.368/0.900
step:303 | loss/acc = 0.356/0.900
step:304 | loss/

step:475 | loss/acc = 0.368/0.850
step:476 | loss/acc = 0.499/0.750
step:477 | loss/acc = 0.442/0.750
step:478 | loss/acc = 0.394/0.850
step:479 | loss/acc = 0.356/0.900
sentiment task:    479: [    1/    2], senti_loss/senti_acc = 0.39819183/0.8300000 
step:480 | loss/acc = 0.408/0.800
step:481 | loss/acc = 0.575/0.650
step:482 | loss/acc = 0.510/0.750
step:483 | loss/acc = 0.613/0.750
step:484 | loss/acc = 0.486/0.700
step:485 | loss/acc = 0.380/0.700
step:486 | loss/acc = 0.399/0.800
step:487 | loss/acc = 0.575/0.700
step:488 | loss/acc = 0.563/0.700
step:489 | loss/acc = 0.494/0.800
sentiment task:    489: [    1/    2], senti_loss/senti_acc = 0.50039234/0.7350000 
step:490 | loss/acc = 0.359/0.850
step:491 | loss/acc = 0.769/0.650
step:492 | loss/acc = 0.460/0.750
step:493 | loss/acc = 0.503/0.650
step:494 | loss/acc = 0.401/0.850
step:495 | loss/acc = 0.386/0.750
step:496 | loss/acc = 0.687/0.700
step:497 | loss/acc = 0.674/0.700
step:498 | loss/acc = 0.626/0.650
step:499 | loss/

step:170 | loss/acc = 0.472/0.850
step:171 | loss/acc = 0.090/1.000
step:172 | loss/acc = 0.135/0.950
step:173 | loss/acc = 0.114/0.950
step:174 | loss/acc = 0.190/0.900
step:175 | loss/acc = 0.065/1.000
step:176 | loss/acc = 0.274/0.850
step:177 | loss/acc = 0.114/0.950
step:178 | loss/acc = 0.060/0.950
step:179 | loss/acc = 0.187/0.950
subjective task:    179: [    0/    2], senti_loss/senti_acc = 0.17018927/0.9350000 
step:180 | loss/acc = 0.445/0.900
step:181 | loss/acc = 0.049/1.000
step:182 | loss/acc = 0.084/0.950
step:183 | loss/acc = 0.491/0.800
step:184 | loss/acc = 0.171/0.900
step:185 | loss/acc = 0.259/0.900
step:186 | loss/acc = 0.242/0.950
step:187 | loss/acc = 0.072/0.950
step:188 | loss/acc = 0.072/0.950
step:189 | loss/acc = 0.163/0.950
subjective task:    189: [    0/    2], senti_loss/senti_acc = 0.20488959/0.9250000 
step:190 | loss/acc = 0.200/0.800
step:191 | loss/acc = 0.118/0.950
step:192 | loss/acc = 0.034/1.000
step:193 | loss/acc = 0.463/0.900
step:194 | los

step:364 | loss/acc = 0.251/0.900
step:365 | loss/acc = 0.177/0.900
step:366 | loss/acc = 0.098/0.950
step:367 | loss/acc = 0.115/1.000
step:368 | loss/acc = 0.178/0.900
step:369 | loss/acc = 0.160/0.950
subjective task:    369: [    0/    2], senti_loss/senti_acc = 0.20711721/0.9200000 
step:370 | loss/acc = 0.040/1.000
step:371 | loss/acc = 0.091/0.950
step:372 | loss/acc = 0.072/1.000
step:373 | loss/acc = 0.496/0.850
step:374 | loss/acc = 0.025/1.000
step:375 | loss/acc = 0.072/1.000
step:376 | loss/acc = 0.280/0.900
step:377 | loss/acc = 0.064/1.000
step:378 | loss/acc = 0.122/0.950
step:379 | loss/acc = 0.092/1.000
subjective task:    379: [    0/    2], senti_loss/senti_acc = 0.13545308/0.9650000 
step:380 | loss/acc = 0.021/1.000
step:381 | loss/acc = 0.105/0.950
step:382 | loss/acc = 0.056/1.000
step:383 | loss/acc = 0.041/1.000
step:384 | loss/acc = 0.099/0.900
step:385 | loss/acc = 0.234/0.900
step:386 | loss/acc = 0.012/1.000
step:387 | loss/acc = 0.097/0.950
step:388 | los

In [22]:
rdm_model = RDM_Model(768, 300, 256, 0.2).cuda()
cm_model = CM_Model(300, 256, 2).cuda()
rdm_classifier = nn.Linear(256, 2).cuda()

/home/hadoop/.conda/envs/py37_torch/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
MTLTrainRDMModel(rdm_model, bert, rdm_classifier,
                     transformer, task_embedding, senti_cls, subj_cls, 
                     senti_train_reader, subj_train_reader, 
                    tt, 20000, new_data_len=[], logger=None, cuda=True, 
                        log_dir="RDMBertTrain")

     0  20000|MTL_Loss:0.69997206, rdm_loss/rdm_acc = 0.72082072/0.0000000 | senti_loss/senti_acc = 0.56915508/0.7000000 | subj_loss/subj_acc = 0.66399979/0.7000000 
     1  20000|MTL_Loss:0.76249111, rdm_loss/rdm_acc = 0.77836663/0.5000000 | senti_loss/senti_acc = 0.64321386/0.7000000 | subj_loss/subj_acc = 0.75476415/0.6000000 
     2  20000|MTL_Loss:0.68839521, rdm_loss/rdm_acc = 0.65440810/1.0000000 | senti_loss/senti_acc = 0.78327168/0.4000000 | subj_loss/subj_acc = 0.86541580/0.4000000 
     3  20000|MTL_Loss:0.63432469, rdm_loss/rdm_acc = 0.60354131/1.0000000 | senti_loss/senti_acc = 0.78165638/0.5000000 | subj_loss/subj_acc = 0.73325985/0.6000000 
     4  20000|MTL_Loss:0.77588555, rdm_loss/rdm_acc = 0.80867416/0.5000000 | senti_loss/senti_acc = 0.58763288/0.7000000 | subj_loss/subj_acc = 0.70182917/0.6000000 
     5  20000|MTL_Loss:0.77552871, rdm_loss/rdm_acc = 0.74261606/0.0000000 | senti_loss/senti_acc = 0.76038471/0.5000000 | subj_loss/subj_acc = 1.05397377/0.3000000 
    